In [13]:
## Load libraries
import pandas as pd
import requests
import json
import time
import os
from datetime import datetime
import scraping
import folium
from folium.plugins import HeatMap

In [14]:
# Load the transaction data
df_falster = pd.read_json('./data/boliga_transaction_history_vacation_homes_lolland_falster.json')
df_sjaelland = pd.read_json('./data/boliga_transaction_history_vacation_homes_sjaelland.json')
# Concatenate the two DataFrames
df = pd.concat([df_falster, df_sjaelland], ignore_index=True)

In [3]:
df.head()

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr,ouId,ouAddress
0,2081894,Gedser Feriepark 51,4874,350000,2024-08-04T22:00:00.000Z,4,Alm. Salg,8333.333,3,42,...,54.584210,11.923484,376,35334,Gedser,NaN,True,283712,71050846,gedser-feriepark-51-4874-gedser
1,0,Stenkløvervej 12,4874,900000,2024-08-04T22:00:00.000Z,4,Fam. Salg,12857.143,4,70,...,54.600700,11.962133,376,33992,Gedser,NaN,True,3126834,131475639,stenkloevervej-12-4874-gedser
2,2115866,Birkely 9,4880,950000,2024-07-31T22:00:00.000Z,4,Alm. Salg,10555.556,4,90,...,54.654774,11.744907,376,12278,Nysted,NaN,True,5433621,396584712,birkely-9-4880-nysted
3,0,"Havnepladsen 2B, 2. tv",4941,1000000,2024-07-30T22:00:00.000Z,4,Alm. Salg,11494.253,3,87,...,54.836502,11.486125,360,9598,Bandholm,NaN,True,100171396,1993672969,havnepladsen-2b-2-tv-4941-bandholm
4,2096034,"Havnepladsen 2C, 2. tv",4941,1000000,2024-07-30T22:00:00.000Z,4,Alm. Salg,12820.513,3,78,...,54.836450,11.486057,360,9602,Bandholm,NaN,True,100171482,1990877815,havnepladsen-2c-2-tv-4941-bandholm


In [26]:

# Filter data for relevant columns
df_filtered = df[['latitude', 'longitude']]

# Drop rows with missing latitude/longitude
df_filtered.dropna(subset=['latitude', 'longitude'], inplace=True)
# Center the map around Sjælland, Denmark
map_center = [55.5, 12.0]

# Create a folium map centered around Sjælland
m = folium.Map(location=map_center, zoom_start=9)

# Convert the filtered dataframe to a list of coordinate tuples
heat_data = df_filtered[['latitude', 'longitude']].values.tolist()

# Create a HeatMap layer and add it to the map
HeatMap(heat_data, radius=8, blur=15, max_zoom=1).add_to(m)
m

/tmp/ipykernel_9215/3807431312.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.dropna(subset=['latitude', 'longitude'], inplace=True)


In [36]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import DivIcon

# Load the transaction data
df_transactions = df

# Convert the municipalityCode in the transaction data to a string
df_transactions['municipalityCode'] = df_transactions['municipalityCode'].astype(str)

# Aggregate transaction data by municipality
df_aggregated = df_transactions.groupby('municipalityCode').size().reset_index(name='transaction_count')

# Load the GeoJSON file with municipalities boundaries
geojson_file = 'data/municipalities.geojson'

# Load the GeoDataFrame
gdf = gpd.read_file(geojson_file)

# Convert the lau_1 column in the GeoDataFrame to a string
gdf['lau_1'] = gdf['lau_1'].astype(str)

# Merge GeoDataFrame with transaction data
gdf = gdf.merge(df_aggregated, left_on='lau_1', right_on='municipalityCode')

# Create a base map centered on Sjælland
m = folium.Map(location=[55.5, 12.0], zoom_start=8)

# Add the choropleth layer to the map
folium.Choropleth(
    geo_data=gdf,
    name='choropleth',
    data=gdf,
    columns=['lau_1', 'transaction_count'],
    key_on='feature.properties.lau_1',
    fill_color='YlOrRd',
    fill_opacity=0.4,
    line_opacity=1.0,
    highlight=True,
    line_color='black'
).add_to(m)

# Add markers for the transaction count
for _, row in gdf.iterrows():
    # Find the center of the municipality
    centroid = row['geometry'].centroid
    # Add a marker with the transaction count

# Add a layer control to toggle layers
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('sjaelland_transactions_choropleth.html')

# Display the map in the notebook (optional)
m
